In [19]:
import re
import numpy as np
import itertools
import pandas as pd
import numpy as np
import re
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import csv 
%matplotlib inline
import matplotlib.pyplot as plt

In [20]:
def create_unique_word_dict(text:list) -> dict:
    """
    Dictionnaire de mot unique, les clés sont les mots et les valeurs sotn les indices des mots
    """
    # Getting all the unique words from our text and sorting them alphabetically
    words = list(set(text))
    words.sort()

    # Creating the dictionary for the unique words
    unique_word_dict = {}
    for i, word in enumerate(words):
        unique_word_dict.update({
            word: i
        })

    return unique_word_dict    

def text_preprocessing(
    text:list,
    punctuations = r'''!()-[]{};:'"`\,<>./?@#$%^&*_“|+=~σ''',
    stop_words=['and', 'a', 'is', 'the', 'in', 'be', 'will', ' http','arxiv','org', 'of']
    )->list:
    """
    Nettoyage du text, stop words et caractères.
    """
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "")

    # Removing words that have numbers in them
    text = re.sub(r'\w*\d\w*', '', text)

    # Removing digits
    text = re.sub(r'[0-9]+', '', text)

    # Cleaning the whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Setting every word to lower
    text = text.lower()

    # Converting all our text to a list 
    text = text.split(' ')

    # Droping empty strings
    text = [x for x in text if x!='']

    # Droping stop words
    text = [x for x in text if x not in stop_words]

    return text

In [21]:
def file_dict(filename):

    #tratement spécial pour les fichiers arxiv
    if filename=='input/arxiv_test.csv':
        
        with open(filename,encoding='utf8') as fichier:
            lines= fichier.readlines()
            texts=[]
            for i in lines:
                texts+=[i.split('|')[1]+' '+ i.split('|')[4]]
    else:
        texts = pd.read_csv(filename,error_bad_lines=False)
        texts = [x for x in texts['text']]#liste des lignes
        
    word_lists = []
    all_text = []

    for text in texts:

    # Cleaning the text
        text = text_preprocessing(text)
        all_text += text
    unique_word_dict = create_unique_word_dict(all_text)

    return(all_text,unique_word_dict)

File est la variable nom du fichier csv à changer

In [23]:
file='input/arxiv_test.csv'
#print(file_dict(file)[0])
#print(file_dict(file)[1])

## Préparation des données

On crée d'abord une matrice A de taille nbre de mot* taille du vocabulaie (matrice de hot vectors)

On récupère Les indices pour créer le jeu de donnée x y 

In [24]:
def create_X(text,dict):
    V=len(text)
    X=np.zeros((V,len(dict)))
    indice=0
    for i in text:
        for cle, valeur in dict.items():
            if i==cle :
                
                X[indice,valeur]=1
        indice+=1
    return X

c'est ici que l'éxecution est longue (pour la création de A et X)

In [25]:
A=create_X(file_dict(file)[0],file_dict(file)[1])

X=[]
for i in range(np.shape(A)[0]):
    for j in range(np.shape(A)[1]):
        if A[i,j]==1 :
            X.append(j)
            
# X listes des indices 

In [26]:
#liste des groupes de 3 mots
def context_list(X,context):
    list=[]
    for i in range(len(X)-context+1):
        list.append(X[i:i+context])
    return list

Voici es données d'entrée

In [27]:
print(context_list(X,3))

[[427, 6, 572], [6, 572, 331], [572, 331, 836], [331, 836, 695], [836, 695, 767], [695, 767, 250], [767, 250, 247], [250, 247, 161], [247, 161, 538], [161, 538, 242], [538, 242, 640], [242, 640, 382], [640, 382, 84], [382, 84, 152], [84, 152, 5], [152, 5, 572], [5, 572, 328], [572, 328, 528], [328, 528, 572], [528, 572, 713], [572, 713, 697], [713, 697, 118], [697, 118, 847], [118, 847, 401], [847, 401, 836], [401, 836, 523], [836, 523, 728], [523, 728, 600], [728, 600, 651], [600, 651, 301], [651, 301, 11], [301, 11, 767], [11, 767, 250], [767, 250, 247], [250, 247, 161], [247, 161, 242], [161, 242, 640], [242, 640, 767], [640, 767, 382], [767, 382, 84], [382, 84, 24], [84, 24, 5], [24, 5, 0], [5, 0, 508], [0, 508, 572], [508, 572, 328], [572, 328, 247], [328, 247, 360], [247, 360, 335], [360, 335, 327], [335, 327, 335], [327, 335, 327], [335, 327, 205], [327, 205, 683], [205, 683, 777], [683, 777, 41], [777, 41, 654], [41, 654, 350], [654, 350, 445], [350, 445, 469], [445, 469, 629],

In [28]:
def vectoword(X,dict): #avec le hot vector je récupère le mot
    c=0
    d=''
    for i in range(len(X)):
        if X[i]==1:
            c=i
    for cle, valeur in dict.items():
        if valeur==c:
            d=cle
    return d
def vectoword2(i,dict): #avec l'indice je récupère le mot
    c=i
    d=''
    for cle, valeur in dict.items():
        if valeur==c:
            d=cle
    return d

In [29]:
print(vectoword2(X[0],file_dict(file)[1]))

lacunary


Pour Y, les sorties, je n'ai besoin que de reprendre les vecteurs de la matrice A correspondant.

In [30]:
#données
n=4
x=context_list(X,3)
Y=A[3:,]
y=[]
for i in range(np.shape(Y)[0]):
    y.append(Y[i])
x=x[:-1]
dict_size=len(file_dict(file)[1])
nb_features=100
h=100

In [31]:
x=np.array(x)
y=np.array(y)